In [102]:
import torch
import torch.nn as nn

# from models.common import *
from yolov7_utils.common import Conv, Concat, MP, SPPCSPC, RepConv, SP # add SP
from nas.supernet.yolo import IDetect
from yolov7_utils.general import make_divisible
from copy import deepcopy

# libraries from ofa
from nas.supernet.dynamic_layers import DynamicConvLayer, DynamicConv2d, DynamicBatchNorm2d
from nas.supernet.search_block import ELAN, ELANBlock, BBoneELAN, HeadELAN, DyConv, TinyELAN, TinyDyConv, ELAN2 # add tinyelan

In [122]:
def parse_supernet(d, ch):  # model_dict, input_channels(3)
    print('\n%3s%18s%3s%10s  %-40s%-30s' % ('', 'from', 'n', 'params', 'module', 'arguments'))
    anchors, nc, gd, gw = d['anchors'], d['nc'], d['depth_multiple'], d['width_multiple']
    na = (len(anchors[0]) // 2) if isinstance(anchors, list) else anchors  # number of anchors
    no = na * (nc + 5)  # number of outputs = anchors * (classes + 5)

    layers, save, c2 = [], [], ch[-1]  # layers, savelist, ch out
    for i, (f, n, m, args) in enumerate(d['backbone'] + d['head']):  # from, number, module, args
        m = eval(m) if isinstance(m, str) else m  # eval strings
        for j, a in enumerate(args):
            try:
                args[j] = eval(a) if isinstance(a, str) else a  # eval strings
            except:
                pass

        n = max(round(n * gd), 1) if n > 1 else n  # depth gain
        if m in [nn.Conv2d, Conv, RepConv, SPPCSPC, DyConv]:
            c1, c2 = ch[f], args[0]
            if c2 != no:  # if not output
                c2 = make_divisible(c2 * gw, 8)

            args = [c1, c2, *args[1:]]
            if m in [SPPCSPC]:
                args.insert(2, n)  # number of repeats
                n = 1
        elif m is BBoneELAN:
            c1, c2 = ch[f], int(args[0]*(args[-1]+1))
            args = [c1, *args]
        elif m is HeadELAN:
            c1, c2 = ch[f], int((args[0]*2) + (args[0]/2 * (args[-1]-1)))
            args = [c1, *args]
        elif m is TinyELAN: # TinyELAN
            c1, c2 = ch[f], args[0]
            args = [c1, c2, *args[1:]]
        elif m is TinyDyConv:
            c1, c2 = args[0], int(args[0]//2)
            if c2 != no:  # if not output
                c2 = make_divisible(c2 * gw, 8)
            args = [c1, c2, *args[1:]]
        elif m is nn.BatchNorm2d:
            args = [ch[f]]
        elif m is Concat:
            c2 = sum([ch[x] for x in f])
        # elif m is Chuncat:
        #     c2 = sum([ch[x] for x in f])
        # elif m is Shortcut:
        #     c2 = ch[f[0]]
        # elif m is Foldcut:
        #     c2 = ch[f] // 2
        elif m in [IDetect]:
            args.append([ch[x] for x in f])
            if isinstance(args[1], int):  # number of anchors
                args[1] = [list(range(args[1] * 2))] * len(f)
        # elif m is ReOrg:
        #     c2 = ch[f] * 4
        # elif m is Contract:
        #     c2 = ch[f] * args[0] ** 2
        # elif m is Expand:
        #     c2 = ch[f] // args[0] ** 2
        else:
            c2 = ch[f]
        print(m)
        m_ = nn.Sequential(*[m(*args) for _ in range(n)]) if n > 1 else m(*args)  # module
        t = str(m)[8:-2].replace('__main__.', '')  # module type
        np = sum([x.numel() for x in m_.parameters()])  # number params
        m_.i, m_.f, m_.type, m_.np = i, f, t, np  # attach index, 'from' index, type, number params
        print('%3s%18s%3s%10.0f  %-40s%-30s' % (i, f, n, np, t, args))  # print
        save.extend(x % i for x in ([f] if isinstance(f, int) else f) if x != -1)  # append to savelist
        layers.append(m_)
        if i == 0:
            ch = []
        ch.append(c2)
    return nn.Sequential(*layers), sorted(save)

TinySuperNet

In [123]:
import yaml
yaml_file = r'C:\Users\Lab13\Documents\GitHub\yolov7-nas-for-TANGO\yaml\yolov7-tinyelan.yaml'
with open(yaml_file) as f:
    yaml = yaml.load(f, Loader=yaml.SafeLoader)

# Define model
ch, nc, anchors = 3, None, None

ch = yaml['ch'] = yaml.get('ch', ch)  # input channels
if nc and nc != yaml['nc']:
    print(f"Overriding model.yaml nc={yaml['nc']} with nc={nc}")
    yaml['nc'] = nc  # override yaml value
if anchors:
    print(f'Overriding model.yaml anchors with anchors={anchors}')
    yaml['anchors'] = round(anchors)  # override yaml value

model, save = parse_supernet(deepcopy(yaml), ch=[ch])


                 from  n    params  module                                  arguments                     
<class 'yolov7_utils.common.Conv'>
  0                -1  1       928  yolov7_utils.common.Conv                [3, 32, 3, 2, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.Conv'>
  1                -1  1     18560  yolov7_utils.common.Conv                [32, 64, 3, 2, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'nas.supernet.search_block.TinyELAN'>
  2                -1  1     22784  nas.supernet.search_block.TinyELAN      [64, 32, 3, 2, LeakyReLU(negative_slope=0.1)]
<class 'nas.supernet.search_block.TinyDyConv'>
  3                -1  1      8320  nas.supernet.search_block.TinyDyConv    [128, 64, 1, 1, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.MP'>
  4                -1  1         0  yolov7_utils.common.MP                  []                            
<class 'nas.supernet.search_block.TinyELAN'>
  5                -1  1     82432  nas

In [124]:
x = torch.randn(1, 3, 64, 64)

y, dt = [], []  # outputs
for i, m in enumerate(model):
    if m.f != -1:  # if not from previous layer
        x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers
    
    x = m(x)  # run
    y.append(x if m.i in save else None)  # save output

In [57]:
import yaml
yaml_file = r'C:\Users\Lab13\Documents\GitHub\yolov7-nas-for-TANGO\yaml\yolov7-tiny.yaml'
#yaml_file = r'C:\Users\Lab13\Documents\GitHub\yolov7-nas-for-TANGO\yaml\yolov7.yaml'
#yaml_file = r'C:\Users\Lab13\Documents\GitHub\yolov7-nas-for-TANGO\yaml\yolov7-tinyelan.yaml'
with open(yaml_file) as f:
    yaml = yaml.load(f, Loader=yaml.SafeLoader)

# Define model
ch, nc, anchors = 3, None, None

ch = yaml['ch'] = yaml.get('ch', ch)  # input channels
if nc and nc != yaml['nc']:
    print(f"Overriding model.yaml nc={yaml['nc']} with nc={nc}")
    yaml['nc'] = nc  # override yaml value
if anchors:
    print(f'Overriding model.yaml anchors with anchors={anchors}')
    yaml['anchors'] = round(anchors)  # override yaml value

In [58]:
model, save = parse_supernet(deepcopy(yaml), ch=[ch])


                 from  n    params  module                                  arguments                     
====== [3] -1 [32, 3, 2, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.Conv'>
  0                -1  1       928  yolov7_utils.common.Conv                [3, 32, 3, 2, None, 1, LeakyReLU(negative_slope=0.1)]
====== [32] -1 [64, 3, 2, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.Conv'>
  1                -1  1     18560  yolov7_utils.common.Conv                [32, 64, 3, 2, None, 1, LeakyReLU(negative_slope=0.1)]
====== [32, 64] -1 [32, 1, 1, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.Conv'>
  2                -1  1      2112  yolov7_utils.common.Conv                [64, 32, 1, 1, None, 1, LeakyReLU(negative_slope=0.1)]
====== [32, 64, 32] -2 [32, 1, 1, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.Conv'>
  3                -2  1      2112  yolov7_utils.common.Conv                [64, 32, 

In [42]:
x = torch.randn(1, 3, 64, 64)

y, dt = [], []  # outputs
for i, m in enumerate(model):
    if m.f != -1:  # if not from previous layer
        x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers
    
    if type(x) == type(list()):
        print(len(x))
    else:
        print(x.shape)
    x = m(x)  # run
    print(m)
    if type(x) == type(list()):
        print(len(x))
    else:
        print(x.shape)
    print('===================================')
    y.append(x if m.i in save else None)  # save output

torch.Size([1, 3, 64, 64])
Conv(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): LeakyReLU(negative_slope=0.1)
)
torch.Size([1, 32, 32, 32])
torch.Size([1, 32, 32, 32])
Conv(
  (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): LeakyReLU(negative_slope=0.1)
)
torch.Size([1, 64, 16, 16])
torch.Size([1, 64, 16, 16])
Conv(
  (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): LeakyReLU(negative_slope=0.1)
)
torch.Size([1, 32, 16, 16])
torch.Size([1, 64, 16, 16])
Conv(
  (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running

In [125]:
from yolov7_utils.common import * 
from nas.supernet.yolo import * 

def forward_once(model, x, runtime_depth, profile=False):
    y, dt = [], []  # outputs
    elan_idx = 0
    for m in model:
        if m.f != -1:  # if not from previous layer
            x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers

        if profile:
            c = isinstance(m, (Detect, IDetect, IAuxDetect, IBin))
            o = thop.profile(m, inputs=(x.copy() if c else x,), verbose=False)[0] / 1E9 * 2 if thop else 0  # FLOPS
            for _ in range(10):
                m(x.copy() if c else x)
            t = time_synchronized()
            for _ in range(10):
                m(x.copy() if c else x)
            dt.append((time_synchronized() - t) * 100)
            print('%10.1f%10.0f%10.1fms %-40s' % (o, m.np, dt[-1], m.type))

        if isinstance(m, ELAN): # 
            depth = runtime_depth[elan_idx]
            elan_idx += 1
            x = m(x, d=depth)
        else:
            x = m(x)  # run
        
        y.append(x if m.i in save else None)  # save output

    if profile:
        print('%.1fms total' % sum(dt))
    return x

In [131]:
input = torch.randn(1, 3, 64, 64)
runtime_depth = [3,2,2,1,5,5,5,3]

forward_once(model, input, runtime_depth)

[tensor([[[[[-6.78405e-01,  7.02849e-02, -4.30802e-01,  ...,  6.40900e-02, -5.05702e-01, -4.89563e-01],
            [-5.82041e-01,  5.33203e-03, -1.08224e+00,  ...,  5.36243e-02, -3.48099e-01, -2.15065e-01],
            [ 2.51822e-01,  2.76155e-01, -4.10832e-02,  ...,  1.42844e-01, -1.29673e-01, -1.03286e-01],
            ...,
            [ 3.37759e-01,  3.15108e-02, -2.30383e-01,  ..., -1.57096e-01,  2.26081e-01, -2.81146e-01],
            [ 7.66103e-02, -9.24093e-02, -2.23878e-01,  ..., -1.34415e-01, -2.03198e-01, -7.19606e-01],
            [ 6.32756e-02, -1.73702e-01, -3.92428e-03,  ..., -1.07633e-01,  1.34479e-02, -2.52676e-01]],
 
           [[ 3.07334e-01, -1.29748e-01, -9.75102e-02,  ..., -1.37271e-01,  8.22126e-02, -2.92656e-01],
            [ 2.00939e-01,  4.87961e-02, -3.12212e-02,  ...,  2.21441e-01,  3.57207e-01, -6.56566e-01],
            [ 4.83077e-01,  2.54820e-01, -6.34505e-02,  ..., -1.75082e-01,  5.28482e-01, -5.69956e-01],
            ...,
            [-2.87946e-02, 

In [127]:
import yaml
yaml_file = r'C:\Users\Lab13\Documents\GitHub\yolov7-nas-for-TANGO\yaml\yolov7-tiny.yaml'
with open(yaml_file) as f:
    yaml = yaml.load(f, Loader=yaml.SafeLoader)

# Define model
ch, nc, anchors = 3, None, None

ch = yaml['ch'] = yaml.get('ch', ch)  # input channels
if nc and nc != yaml['nc']:
    print(f"Overriding model.yaml nc={yaml['nc']} with nc={nc}")
    yaml['nc'] = nc  # override yaml value
if anchors:
    print(f'Overriding model.yaml anchors with anchors={anchors}')
    yaml['anchors'] = round(anchors)  # override yaml value

In [128]:
model, save = parse_supernet(deepcopy(yaml), ch=[ch])


                 from  n    params  module                                  arguments                     
<class 'yolov7_utils.common.Conv'>
  0                -1  1       928  yolov7_utils.common.Conv                [3, 32, 3, 2, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.Conv'>
  1                -1  1     18560  yolov7_utils.common.Conv                [32, 64, 3, 2, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.Conv'>
  2                -1  1      2112  yolov7_utils.common.Conv                [64, 32, 1, 1, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.Conv'>
  3                -2  1      2112  yolov7_utils.common.Conv                [64, 32, 1, 1, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.Conv'>
  4                -1  1      9280  yolov7_utils.common.Conv                [32, 32, 3, 1, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.Conv'>
  5                -1  1    

In [129]:
from yolov7_utils.common import * 
from nas.supernet.yolo import * 

def forward_once(model, x, runtime_depth, profile=False):
    y, dt = [], []  # outputs
    elan_idx = 0
    for m in model:
        if m.f != -1:  # if not from previous layer
            x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers

        if profile:
            c = isinstance(m, (Detect, IDetect, IAuxDetect, IBin))
            o = thop.profile(m, inputs=(x.copy() if c else x,), verbose=False)[0] / 1E9 * 2 if thop else 0  # FLOPS
            for _ in range(10):
                m(x.copy() if c else x)
            t = time_synchronized()
            for _ in range(10):
                m(x.copy() if c else x)
            dt.append((time_synchronized() - t) * 100)
            print('%10.1f%10.0f%10.1fms %-40s' % (o, m.np, dt[-1], m.type))

        if isinstance(m, ELAN): # 
            depth = runtime_depth[elan_idx]
            elan_idx += 1
            x = m(x, d=depth)
        else:
            x = m(x)  # run
        
        y.append(x if m.i in save else None)  # save output

    if profile:
        print('%.1fms total' % sum(dt))
    return x

In [130]:
input = torch.randn(1, 3, 64, 64)
runtime_depth = [3,2,2,1,5,5,5,3]

forward_once(model, input, runtime_depth)

[tensor([[[[[-2.59217e-01,  2.73302e-02, -3.36205e-01,  ..., -3.50571e-01,  8.29943e-02, -2.72446e-01],
            [-2.30617e-01, -2.07908e-01, -5.75092e-01,  ..., -1.77278e-01, -5.21586e-01, -2.10560e-01],
            [ 2.81397e-01,  5.93484e-02, -3.56804e-01,  ..., -5.28644e-01, -4.84401e-01, -4.36117e-02],
            ...,
            [ 6.48356e-01, -3.20504e-01,  1.79600e-01,  ...,  1.64460e-01, -1.15900e-01, -2.82422e-01],
            [ 3.23149e-01, -2.54507e-01,  1.34910e-02,  ..., -2.02645e-01,  3.22780e-01, -5.07045e-01],
            [ 3.37949e-01, -3.57915e-01, -2.75098e-01,  ..., -4.73626e-01,  1.72807e-01, -1.24071e-01]],
 
           [[ 2.17980e-01, -8.48220e-02, -5.48084e-01,  ..., -2.69541e-01,  6.99845e-01,  6.71193e-02],
            [ 8.13586e-01,  1.66187e-02, -2.64918e-01,  ...,  2.62164e-02, -1.87421e-02, -1.87663e-01],
            [ 5.29423e-01, -5.20814e-01, -5.69818e-01,  ...,  2.86344e-02,  1.04055e-01, -6.66789e-01],
            ...,
            [-2.63090e-01, 

===========================

In [118]:
import yaml
#yaml_file = r'C:\Users\Lab13\Documents\GitHub\yolov7-nas-for-TANGO\yaml\yolov7-tiny.yaml'
#yaml_file = r'C:\Users\Lab13\Documents\GitHub\yolov7-nas-for-TANGO\yaml\yolov7.yaml'
yaml_file = r'C:\Users\Lab13\Documents\GitHub\yolov7-nas-for-TANGO\yaml\yolov7-tinyelan.yaml'
with open(yaml_file) as f:
    yaml = yaml.load(f, Loader=yaml.SafeLoader)

# Define model
ch, nc, anchors = 3, None, None

ch = yaml['ch'] = yaml.get('ch', ch)  # input channels
if nc and nc != yaml['nc']:
    print(f"Overriding model.yaml nc={yaml['nc']} with nc={nc}")
    yaml['nc'] = nc  # override yaml value
if anchors:
    print(f'Overriding model.yaml anchors with anchors={anchors}')
    yaml['anchors'] = round(anchors)  # override yaml value

model, save = parse_supernet(deepcopy(yaml), ch=[ch])


                 from  n    params  module                                  arguments                     
====== [3] -1 [32, 3, 2, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.Conv'>
  0                -1  1       928  yolov7_utils.common.Conv                [3, 32, 3, 2, None, 1, LeakyReLU(negative_slope=0.1)]
====== [32] -1 [64, 3, 2, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.Conv'>
  1                -1  1     18560  yolov7_utils.common.Conv                [32, 64, 3, 2, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'nas.supernet.search_block.TinyELAN'>
  2                -1  1     22784  nas.supernet.search_block.TinyELAN      [64, 32, 3, 2, LeakyReLU(negative_slope=0.1)]
<class 'nas.supernet.search_block.TinyDyConv'>
  3                -1  1      8320  nas.supernet.search_block.TinyDyConv    [128, 64, 1, 1, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.MP'>
  4                -1  1         0  yolov7_utils.common.MP  

In [119]:
x = torch.randn(1, 3, 64, 64)

y, dt = [], []  # outputs
for i, m in enumerate(model):
    if m.f != -1:  # if not from previous layer
        x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers
    
    if type(x) == type(list()):
        print(len(x))
    else:
        print(x.shape)
    print(m)
    x = m(x)  # run
    if type(x) == type(list()):
        print(len(x))
    else:
        print(x.shape)
    print('===================================')
    y.append(x if m.i in save else None)  # save output

torch.Size([1, 3, 64, 64])
Conv(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): LeakyReLU(negative_slope=0.1)
)
torch.Size([1, 32, 32, 32])
torch.Size([1, 32, 32, 32])
Conv(
  (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): LeakyReLU(negative_slope=0.1)
)
torch.Size([1, 64, 16, 16])
torch.Size([1, 64, 16, 16])
TinyELAN(
  (act): LeakyReLU(negative_slope=0.1)
  (layers): Sequential(
    (0): Conv(
      (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): LeakyReLU(negative_slope=0.1)
    )
    (1): Conv(
      (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNor